In [20]:
import sentinel1decoder
import pandas as pd
import numpy as np
import logging
import math
import cmath
import struct
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.interpolate import interp1d
from skimage.transform import rescale, resize, downscale_local_mean
import os
from pathlib import Path

numpy_folder = '/home/roberto/PythonProjects/SSFocus/Data/RAW/SM/numpy'

In [2]:
inputfile = "/home/roberto/PythonProjects/SSFocus/Data/RAW/SM/dat/s1a-s1-raw-s-vv-20200509t183227-20200509t183238-032492-03c34a.dat"
l0file = sentinel1decoder.Level0File(inputfile)

In [3]:
l0file.packet_metadata


Packet Version Number  Packet Type  \
Azimuth Block Number Packet Number                                       
1                    0                                  0            0   
                     1                                  0            0   
                     2                                  0            0   
                     3                                  0            0   
                     4                                  0            0   
...                                                   ...          ...   
15                   19521                              0            0   
                     19522                              0            0   
                     19523                              0            0   
                     19524                              0            0   
                     19525                              0            0   

                                    Secondary Header Flag  PID  PCAT  \
Azimuth Block Number Packet Number                                     
1                    0                                  1   65    12   
                     1                                  1   65    12   
                     2                                  1   65    12   
                     3                                  1   65    12   
                     4                                  1   65    12   
...                                                   ...  ...   ...   
15                   19521                              1   65    12   
                     19522                              1   65    12   
                     19523                              1   65    12   
                     19524                              1   65    12   
                     19525                              1   65    12   

                                    Sequence Flags  Packet Sequence Count  \
Azimuth Block Number Packet Number                                          
1                    0                           3                      0   
                     1                           3                      1   
                     2                           3                      2   
                     3                           3                      3   
                     4                           3                      4   
...                                            ...                    ...   
15                   19521                       3                      1   
                     19522                       3                      2   
                     19523                       3                      3   
                     19524                       3                      4   
                     19525                       3                      5   

                                    Packet Data Length  Coarse Time  \
Azimuth Block Number Packet Number                                    
1                    0                           32394   1273084365   
                     1                           32394   1273084365   
                     2                           32394   1273084365   
                     3                           32394   1273084365   
                     4                           32394   1273084365   
...                                                ...          ...   
15                   19521                       32394   1273084376   
                     19522                       32394   1273084376   
                     19523                       32394   1273084376   
                     19524                       32394   1273084376   
                     19525                       32394   1273084376   

                                    Fine Time  ...       SWL  SAS SSB Flag  \
Azimuth Block Number Packet Number             ...                           
1                    0               0.807060  ...  0.

In [4]:
l0file.ephemeris


,X-axis position ECEF,Y-axis position ECEF,Z-axis position ECEF,X-axis velocity ECEF,Y-axis velocity ECEF,Z-axis velocity ECEF,POD Solution Data Timestamp,Q0 Attitude Quaternion,Q1 Attitude Quaternion,Q2 Attitude Quaternion,Q3 Attitude Quaternion,Omega-X Angular Rate,Omega-Y Angular Rate,Omega-Z Angular Rate,Attitude Data Timestamp
0,6.208562e+06,-1.308893e+06,3.124293e+06,-3555.352051,-940.613831,6646.937988,1.273084e+09,0.461479,0.718545,-0.355371,0.380055,0.000069,-0.000931,-0.000504,1.273084e+09
1,6.208562e+06,-1.308893e+06,3.124293e+06,-3555.352051,-940.613831,6646.937988,1.273084e+09,0.461479,0.718545,-0.355371,0.380055,0.000069,-0.000931,-0.000504,1.273084e+09
2,6.208562e+06,-1.308893e+06,3.124293e+06,-3555.352051,-940.613831,6646.937988,1.273084e+09,0.461479,0.718545,-0.355371,0.380055,0.000069,-0.000931,-0.000504,1.273084e+09
3,6.208562e+06,-1.308893e+06,3.124293e+06,-3555.352051,-940.613831,6646.937988,1.273084e+09,0.461479,0.718545,-0.355371,0.380055,0.000069,-0.000931,-0.000504,1.273084e+09
4,6.208562e+06,-1.308893e+06,3.124293e+06,-3555.352051,-940.613831,6646.937988,1.273084e+09,0.461479,0.718545,-0.355371,0.380055,0.000069,-0.000931,-0.000504,1.273084e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,6.172654e+06,-1.318200e+06,3.190584e+06,-3626.133545,-920.646118,6611.267578,1.273084e+09,0.460518,0.721366,-0.355572,0.375664,0.000072,-0.000932,-0.000505,1.273084e+09
297,6.172654e+06,-1.318200e+06,3.190584e+06,-3626.133545,-920.646118,6611.267578,1.273084e+09,0.460518,0.721366,-0.355572,0.375664,0.000072,-0.000932,-0.000505,1.273084e+09
298,6.172654e+06,-1.318200e+06,3.190584e+06,-3626.133545,-920.646118,6611.267578,1.273084e+09,0.460518,0.721366,-0.355572,0.375664,0.000072,-0.000932,-0.000505,1.273084e+09
299,6.169024e+06,-1.319119e+06,3.197193e+06,-3633.188232,-918.637817,6607.659180,1.273084e+09,0.460421,0.721646,-0.355591,0.375225,0.000070,-0.000931,-0.000505,1.273084e+09


In [18]:
selected_burst = 8
selection = l0file.get_burst_metadata(selected_burst)
selection


,Packet Version Number,Packet Type,Secondary Header Flag,PID,PCAT,Sequence Flags,Packet Sequence Count,Packet Data Length,Coarse Time,Fine Time,...,SWL,SAS SSB Flag,Polarisation,Temperature Compensation,Calibration Mode,Tx Pulse Number,Signal Type,Swap Flag,Swath Number,Number of Quads
Packet Number,,,,,,,,,,,,,,,,,,,,,
408,0,0,1,65,12,3,24,16858,1273084366,0.089317,...,0.000259,0,6,3,0,0,0,0,0,12890
409,0,0,1,65,12,3,25,16862,1273084366,0.089851,...,0.000259,0,6,3,0,0,0,0,0,12890
410,0,0,1,65,12,3,26,16914,1273084366,0.090385,...,0.000259,0,6,3,0,0,0,0,0,12890
411,0,0,1,65,12,3,27,16882,1273084366,0.090919,...,0.000259,0,6,3,0,0,0,0,0,12890
412,0,0,1,65,12,3,28,16878,1273084366,0.091469,...,0.000259,0,6,3,0,0,0,0,0,12890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19113,0,0,1,65,12,3,41,16738,1273084376,0.088493,...,0.000259,0,6,3,0,0,0,0,0,12890
19114,0,0,1,65,12,3,42,16762,1273084376,0.089027,...,0.000259,0,6,3,0,0,0,0,0,12890
19115,0,0,1,65,12,3,43,16750,1273084376,0.089561,...,0.000259,0,6,3,0,0,0,0,0,12890


In [19]:
# Decode the IQ data
radar_data = l0file.get_burst_data(selected_burst)

## save data:

In [21]:
L0_name = Path(inputfile).stem + '.npy'
np.save(os.path.join(numpy_folder, L0_name), radar_data)

In [22]:
from SARProcessor.focus import fft_radar_rg_az


